## Extract Historical Stock Data from Yahoo Finance (openBB)

### Import Libraries

In [1]:
import sys
import os
from py.fetch_price_history import fetch_price_history_openbb      

import pandas as pd
import logging
from dotenv import load_dotenv

load_dotenv()

from openbb import obb
import polars as pl
obb.user.credentials.fmp_api_key = os.getenv("FMP_API_KEY")

### Define Variables

In [2]:
from pandas.tseries.offsets import BDay
from datetime import datetime, timedelta

# Time Range adjustment
end_date = (datetime.today() - BDay(1)).to_pydatetime()  # Last business day
start_date = end_date - timedelta(days=10*365)  # 10 years of data

# Convert datetime objects to Unix timestamps (seconds since Jan 1, 1970)
start_timestamp = int(start_date.timestamp())
end_timestamp = int(end_date.timestamp())

# Print the date range
days_difference = (end_date - start_date).days
print(f"Date Range: {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")
print(f"Time span: {days_difference} days ({days_difference/365:.2f} years)")

Date Range: 2015-06-12 to 2025-06-09
Time span: 3650 days (10.00 years)


### Stocks

In [3]:
tickers_file = 'tickers_sp_500.txt'
with open(tickers_file, 'r') as f:
    tickers = [line.strip() for line in f if line.strip()]

# Replace '.' with '-' for Yahoo Finance compatibility
tickers = [t.replace('.', '-') for t in tickers]
tickers = list(dict.fromkeys(tickers))  # Remove duplicates

###########################################################
# DAILY DATA
###########################################################

daily_output = 'daily_stock_quotes.csv'

df_daily, df_daily_failed = fetch_price_history_openbb(
    tickers, 
    start_date, 
    end_date,
    data_file=daily_output,
    interval='1d',                 # options: ['1d', '1w', '1M']
    provider='fmp',                # options: ['fmp', 'yfinance']
    row_threshold_pct=0.05,        # Filter out rows with fewer than 5% of columns containing values
    column_threshold=0.2,          # Filter out columns with less than 20% of values
    validate_recent_data=True,     # Enable recent data validation
    recent_data_percentage=0.2     # Check the last 20% of rows
)

###########################################################
# MONTHLY DATA
###########################################################

monthly_output = 'monthly_stock_quotes.csv'

df = (df_daily.set_index(pd.to_datetime(df_daily.pop('Date')))
      if 'Date' in df_daily.columns else df_daily.copy())
df.index = pd.to_datetime(df.index)              
(df.resample('MS').last()
   .reset_index()
   .to_csv(monthly_output, index=False))

📂 Reading existing data from: daily_stock_quotes.csv
🗓 Existing data: 2015-05-19 to 2025-06-09
✅ Found 500 tickers, ❌ Missing 3 tickers
⏳ Fetching data using provider: fmp...
Will fetch 3 missing tickers from 2015-05-19 to 2025-06-09
Using FMP batch processing for 503 tickers
Fetching history for 3 missing tickers...
Processing attempt 1/2
Fetching batch of 3 tickers...
Retrieved data for 3 of 3 missing tickers
⚠️ Column 'GEV' missing 205 values (40.5%) in recent data
⚠️ Column 'SOLV' missing 204 values (40.3%) in recent data
⚠️ Column 'VLTO' missing 80 values (15.8%) in recent data
🔄 Fetching missing recent data for 3 tickers...
Processing attempt 1/2
Fetching batch of 3 tickers...
✅ Updated missing recent data for 3 tickers
⚠️ Removing 3 columns with < 20.0% data or < 5 values:
   GEV (301 values, 11.90%), SOLV (302 values, 11.94%), VLTO (426 values, 16.84%)
💾 Saved updated data to daily_stock_quotes.csv


### Bonds

In [4]:
tickers_file = 'tickers_bond.txt'
with open(tickers_file, 'r') as f:
    tickers = [line.strip() for line in f if line.strip()]

# Replace '.' with '-' for Yahoo Finance compatibility
tickers = [t.replace('.', '-') for t in tickers]
tickers = list(dict.fromkeys(tickers))  # Remove duplicates

###########################################################
# DAILY DATA
###########################################################

daily_output = 'daily_bond_quotes.csv'

bonds_daily_df, failed_daily = fetch_price_history_openbb(
    tickers, 
    start_date, 
    end_date,
    data_file=daily_output,
    interval='1d',                 # options: ['1d', '1w', '1M']
    provider='fmp',                # options: ['fmp', 'yfinance']
    row_threshold_pct=0.05,        # Filter out rows with fewer than 5% of columns containing values
    column_threshold=0.2,          # Filter out columns with less than 20% of values
    validate_recent_data=True,     # Enable recent data validation
    recent_data_percentage=0.2     # Check the last 20% of rows
)

###########################################################
# MONTHLY DATA
###########################################################

monthly_output = 'monthly_bond_quotes.csv'

bonds_monthly_prices = (bonds_daily_df.set_index(pd.to_datetime(bonds_daily_df.pop('Date')))
      if 'Date' in bonds_daily_df.columns else bonds_daily_df.copy())
bonds_monthly_prices.index = pd.to_datetime(bonds_monthly_prices.index)              
bonds_monthly_prices = (bonds_monthly_prices.resample('MS').last()
   .reset_index()
   .rename(columns={'index': 'Date'}))

# Save to CSV
bonds_monthly_prices.to_csv(monthly_output, index=False)

display(bonds_monthly_prices.head())

📂 Reading existing data from: daily_bond_quotes.csv
🗓 Existing data: 2015-05-19 to 2025-06-09
✅ Found 101 tickers, ❌ Missing 46 tickers
⏳ Fetching data using provider: fmp...
Will fetch 46 missing tickers from 2015-05-19 to 2025-06-09
Using FMP batch processing for 147 tickers
Fetching history for 46 missing tickers...
Processing attempt 1/2
Fetching batch of 46 tickers...
Retrieved data for 27 of 46 missing tickers
⚠️ Column 'AGRH' missing 1 values (0.2%) in recent data
⚠️ Column 'BMOPX' missing 506 values (100.0%) in recent data
⚠️ Column 'CSHP' missing 281 values (55.5%) in recent data
⚠️ Column 'IBCA' missing 453 values (89.5%) in recent data
⚠️ Column 'IBDY' missing 14 values (2.8%) in recent data
⚠️ Column 'IBDZ' missing 246 values (48.6%) in recent data
⚠️ Column 'IBGA' missing 258 values (51.0%) in recent data
⚠️ Column 'IBGB' missing 454 values (89.7%) in recent data
⚠️ Column 'IBGL' missing 454 values (89.7%) in recent data
⚠️ Column 'IBHJ' missing 14 values (2.8%) in recent 

,Date,AGG,AGIH,AGRH,AGZ,BEMB,BGRN,BMOIX,BYLD,CEMB,...,SUB,SUSB,SUSC,TFLO,TIP,TLH,TLT,TLTW,USHY,USIG
0,2015-05-01,85.03,NaN,NaN,92.41,NaN,NaN,7.83,17.45,32.96,...,93.31,NaN,NaN,41.86,87.28,105.66,94.83,NaN,NaN,39.68
1,2015-06-01,84.11,NaN,NaN,92.06,NaN,NaN,7.75,17.23,32.12,...,93.44,NaN,NaN,41.83,86.39,103.55,90.97,NaN,NaN,39.09
2,2015-07-01,84.84,NaN,NaN,92.44,NaN,NaN,7.80,17.36,32.32,...,93.75,NaN,NaN,41.74,86.85,106.14,95.10,NaN,NaN,39.42
3,2015-08-01,84.55,NaN,NaN,92.57,NaN,NaN,7.79,17.27,31.66,...,93.52,NaN,NaN,41.72,86.03,105.57,94.44,NaN,NaN,39.02
4,2015-09-01,85.24,NaN,NaN,93.05,NaN,NaN,7.84,17.21,30.80,...,93.84,NaN,NaN,41.78,85.58,107.51,96.30,NaN,NaN,39.37


### Benchmarks

In [5]:
tickers_file = 'tickers_benchmark.txt'
with open(tickers_file, 'r') as f:
    tickers = [line.strip() for line in f if line.strip()]

# Replace '.' with '-' for Yahoo Finance compatibility
tickers = [t.replace('.', '-') for t in tickers]
tickers = list(dict.fromkeys(tickers))  # Remove duplicates

###########################################################
# DAILY DATA
###########################################################

daily_output = 'daily_benchmark_quotes.csv'

df_daily, df_daily_failed = fetch_price_history_openbb(
    tickers, 
    start_date, 
    end_date,
    data_file=daily_output,
    interval='1d',                 # options: ['1d', '1w', '1M']
    provider='fmp',                # options: ['fmp', 'yfinance']
    row_threshold_pct=0.05,        # Filter out rows with fewer than 5% of columns containing values
    column_threshold=0.2,          # Filter out columns with less than 20% of values
    validate_recent_data=True,     # Enable recent data validation
    recent_data_percentage=0.2     # Check the last 20% of rows
)

###########################################################
# MONTHLY DATA
###########################################################

monthly_output = 'monthly_benchmark_quotes.csv'

df = (df_daily.set_index(pd.to_datetime(df_daily.pop('Date')))
      if 'Date' in df_daily.columns else df_daily.copy())
df.index = pd.to_datetime(df.index)              
(df.resample('MS').last()
   .reset_index()
   .to_csv(monthly_output, index=False))

📂 Reading existing data from: daily_benchmark_quotes.csv
🗓 Existing data: 2015-05-19 to 2025-06-09
✅ Found 544 tickers, ❌ Missing 0 tickers
✅ Data is already up-to-date and complete
⚠️ Column 'JDIV' missing 331 values (65.4%) in recent data
🔄 Fetching missing recent data for 1 tickers...
Processing attempt 1/2
Fetching batch of 1 tickers...
💾 Saved updated data with fixed recent values to daily_benchmark_quotes.csv
✅ Updated missing recent data for 1 tickers


### Treasury Rates

In [6]:
tickers_file = 'tickers_treasury.txt'
with open(tickers_file, 'r') as f:
    tickers = [line.strip() for line in f if line.strip()]

# Replace '.' with '-' for Yahoo Finance compatibility
tickers = [t.replace('.', '-') for t in tickers]
tickers = list(dict.fromkeys(tickers))  # Remove duplicates

###########################################################
# DAILY DATA
###########################################################

daily_output = 'daily_treasury_rates.csv'

df_daily, df_daily_failed = fetch_price_history_openbb(
    tickers, 
    start_date, 
    end_date,
    data_file=daily_output,
    interval='1d',                 # options: ['1d', '1w', '1M']
    provider='fmp',                # options: ['fmp', 'yfinance']
    row_threshold_pct=0.05,        # Filter out rows with fewer than 5% of columns containing values
    column_threshold=0.2,          # Filter out columns with less than 20% of values
    validate_recent_data=True,     # Enable recent data validation
    recent_data_percentage=0.2     # Check the last 20% of rows
)

###########################################################
# MONTHLY DATA
###########################################################

monthly_output = 'monthly_treasury_rates.csv'

df = (df_daily.set_index(pd.to_datetime(df_daily.pop('Date')))
      if 'Date' in df_daily.columns else df_daily.copy())
df.index = pd.to_datetime(df.index)              
(df.resample('MS').last()
   .reset_index()
   .to_csv(monthly_output, index=False))

📂 Reading existing data from: daily_treasury_rates.csv
🗓 Existing data: 2015-05-19 to 2025-06-09
✅ Found 4 tickers, ❌ Missing 0 tickers
✅ Data is already up-to-date and complete


### Sectors

In [7]:
sectors = [
    'XLE',
    'CLF',
    'XLF',
    'GDX'
]